# Computation of magnification factor using pycortex
__To do :__
- [x] adapt it to vertice analysis

In [1]:
# Stop warnings
import warnings
warnings.filterwarnings("ignore")

# General imports
import cortex
import importlib
import json
import numpy as np
import os
import sys
sys.path.append("{}/../../../utils".format(os.getcwd()))
from pycortex_utils import draw_cortex, set_pycortex_config_file, load_surface_pycortex,lire_fichier_npz
import nibabel as nb

# Define analysis parameters
with open('../../../settings.json') as f:
    json_s = f.read()
    analysis_info = json.loads(json_s)
tasks = analysis_info["task_names"]
task = tasks[2]
rois = analysis_info["rois"]
vert_dist_th = analysis_info['vertex_pcm_rad']
formats = analysis_info['formats']

# debug
formats = ['fsnative']
rois = ['V1']

# Inputs
# main_dir = '/home/mszinte/disks/meso_S/data'
main_dir = '/Users/uriel/disks/meso_shared/'
project_dir = 'RetinoMaps'
subject = 'sub-01'
deriv_fn_label = 'avg-gridfit'
model = 'gauss'

# Set pycortex db and colormaps
cortex_dir = "{}/{}/derivatives/pp_data/cortex".format(main_dir, project_dir)
set_pycortex_config_file(cortex_dir)
importlib.reload(cortex)


for format_, pycortex_subject in zip(formats, [subject, 'sub-170k']):
    
    # define directories and fn
    prf_dir = "{}/{}/derivatives/pp_data/{}/{}/prf".format(main_dir, project_dir, 
                                                           subject, format_)
    fit_dir = "{}/fit".format(prf_dir)
    prf_deriv_dir = "{}/prf_derivatives".format(prf_dir)

    if format_ == 'fsnative':
        deriv_avg_fn_L = '{}/{}_task-{}_hemi-L_fmriprep_dct_avg_prf-deriv_{}_gridfit.func.gii'.format(
            prf_deriv_dir, subject, task, model)
        deriv_avg_fn_R = '{}/{}_task-{}_hemi-R_fmriprep_dct_avg_prf-deriv_{}_gridfit.func.gii'.format(
            prf_deriv_dir, subject, task, model)
        deriv_mat = load_surface_pycortex(L_fn=deriv_avg_fn_L, 
                                          R_fn=deriv_avg_fn_R,)
        
    elif format_ == '170k':
        deriv_avg_fn = '{}/{}_task-{}_fmriprep_dct_avg_prf-deriv_{}_gridfit.dtseries.nii'.format(
            prf_deriv_dir, subject, task, model)
        deriv_mat = load_surface_pycortex(brain_fn=deriv_avg_fn)
        save_svg = False


In [3]:
# get surfaces for each hemisphere
surfs = [cortex.polyutils.Surface(*d) for d in cortex.db.get_surf(subject, "flat")]
surf_lh, surf_rh = surfs[0], surfs[1]
# get the vertices number per hemisphere
lh_vert_num, rh_vert_num = surf_lh.pts.shape[0], surf_rh.pts.shape[0]
vert_num = lh_vert_num + rh_vert_num

# get a dicst with the surface vertices contained in each ROI
roi_verts_dict = cortex.utils.get_roi_verts(subject, mask=False)
#### TO REPLACE BY YOUR NEW FUNCTION TO GET ROIS FROM NPZ IN CASE OF SUB-17K

# derivatives settings
rsq_idx, ecc_idx, polar_real_idx, polar_imag_idx , size_idx, \
    amp_idx, baseline_idx, x_idx, y_idx, hrf_1_idx, hrf_2_idx = \
    0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
if model == 'gauss':
    loo_rsq_idx = 11
elif model == 'dn':
    srf_amp_idx = 11
    sff_size = 12
    neural_baseline_idx = 13
    surround_baseline = 14
    loo_rsq_idx = 15
elif model == 'css':
    n_idx = 11
    loo_rsq_idx = 12

# parameters
vert_rsq_data = deriv_mat[rsq_idx, ...]
vert_x_data = deriv_mat[x_idx, ...]
vert_y_data = deriv_mat[y_idx, ...]
vert_size_data = deriv_mat[size_idx, ...]
vert_ecc_data = deriv_mat[ecc_idx, ...]

# create empty results
vert_cm = np.zeros(vert_num)*np.nan

In [8]:
roi_verts_dict

{'mPCS': array([103034, 103035, 103036, ..., 385000, 459677, 459678]),
 'sPCS': array([ 93679,  93680,  93692, ..., 382190, 382201, 382202]),
 'iPCS': array([114698, 114710, 114723, ..., 400664, 400665, 400666]),
 'sIPS': array([], dtype=float64),
 'iIPS': array([], dtype=float64),
 'VO': array([], dtype=float64),
 'LO': array([], dtype=float64),
 'hMT+': array([ 18352,  18378,  18379, ..., 289886, 289906, 289931]),
 'V3AB': array([  2216,   2230,   2231, ..., 249424, 249437, 249451]),
 'V3': array([   199,    200,    201, ..., 276988, 276989, 276990]),
 'V2': array([     0,      1,      2, ..., 276862, 276863, 276907]),
 'V1': array([    16,    126,    127, ..., 271894, 271895, 271913])}

In [8]:
[rsq_idx, ecc_idx, size_idx, x_idx, y_idx]

[0, 1, 4, 7, 8]

In [6]:
for roi in rois:
    # find ROI vertex
    roi_vert_lh_idx = roi_verts_dict[roi][roi_verts_dict[roi]<lh_vert_num]
    roi_vert_rh_idx = roi_verts_dict[roi][roi_verts_dict[roi]>=lh_vert_num]
    roi_surf_lh_idx = roi_vert_lh_idx
    roi_surf_rh_idx = roi_vert_rh_idx-lh_vert_num

    # get mean distance of surounding vertices included in threshold
    vert_lh_rsq, vert_lh_size = vert_rsq_data[:lh_vert_num], vert_size_data[:lh_vert_num]
    vert_lh_x, vert_lh_y = vert_x_data[:lh_vert_num], vert_y_data[:lh_vert_num]
    vert_rh_rsq, vert_rh_size = vert_rsq_data[lh_vert_num:], vert_size_data[lh_vert_num:]
    vert_rh_x, vert_rh_y = vert_x_data[lh_vert_num:], vert_y_data[lh_vert_num:]

    for hemi in ['lh','rh']:
        if hemi == 'lh':
            surf = surf_lh
            roi_vert_idx, roi_surf_idx = roi_vert_lh_idx, roi_surf_lh_idx
            vert_rsq, vert_x, vert_y, vert_size = vert_lh_rsq, vert_lh_x, vert_lh_y, vert_lh_size
        elif hemi == 'rh':
            surf = surf_rh
            roi_vert_idx, roi_surf_idx = roi_vert_rh_idx, roi_surf_rh_idx
            vert_rsq, vert_x, vert_y, vert_size = vert_rh_rsq, vert_rh_x, vert_rh_y, vert_rh_size

        desc = 'ROI -> {} / Hemisphere -> {}'.format(roi, hemi)
        print(desc)
        for i, (vert_idx, surf_idx) in enumerate(zip(roi_vert_idx, roi_surf_idx)):

            if vert_rsq[surf_idx] > 0:

                # get geodesic distances (mm)
                try :
                    geo_patch = surf.get_geodesic_patch(radius=vert_dist_th, vertex=surf_idx)
                except Exception as e:
                    print("Vertex #{}: error: {} within {} mm".format(vert_idx, e, vert_dist_th))
                    geo_patch['vertex_mask'] = np.zeros(surf.pts.shape[0]).astype(bool)
                    geo_patch['geodesic_distance'] = []

                vert_dist_th_idx  = geo_patch['vertex_mask']
                vert_dist_th_dist = np.ones_like(vert_dist_th_idx)*np.nan
                vert_dist_th_dist[vert_dist_th_idx] = geo_patch['geodesic_distance']

                # exclude vextex out of roi
                vert_dist_th_not_in_roi_idx = [idx for idx in np.where(vert_dist_th_idx)[0] if idx not in roi_surf_idx]
                vert_dist_th_idx[vert_dist_th_not_in_roi_idx] = False
                vert_dist_th_dist[vert_dist_th_not_in_roi_idx] = np.nan

                if np.sum(vert_dist_th_idx) > 0:

                    # compute average geodesic distance excluding distance to itself (see [1:])
                    vert_geo_dist_avg = np.nanmean(vert_dist_th_dist[1:])

                    # get prf parameters of vertices in geodesic distance threshold
                    vert_ctr_x, vert_ctr_y = vert_x[surf_idx], vert_y[surf_idx]
                    vert_dist_th_idx[surf_idx] = False
                    vert_srd_x, vert_srd_y = np.nanmean(vert_x[vert_dist_th_idx]), np.nanmean(vert_y[vert_dist_th_idx])

                    # compute prf center suround distance (deg)
                    vert_prf_dist = np.sqrt((vert_ctr_x - vert_srd_x)**2 + (vert_ctr_y - vert_srd_y)**2)

                    # compute cortical magnification in mm/deg (surface distance / pRF positon distance)
                    vert_cm[vert_idx] = vert_geo_dist_avg/vert_prf_dist


ROI -> V1 / Hemisphere -> lh
ROI -> V1 / Hemisphere -> rh


In [24]:
deriv_mat_new = np.zeros((deriv_mat.shape[0]+1, deriv_mat.shape[1]))*np.nan
deriv_mat_new[0:-1,...] = deriv_mat
deriv_mat_new[-1,...] = vert_cm
### TO SAVE WITH NEW save_surface_pycortex WHICH WILL SPLIT DATA FOR GIFTI BUT NOT CIFTI

In [1]:
roi_surf_lh_idx

NameError: name 'roi_surf_lh_idx' is not defined